# 01 Demo Environment | 01 Create Azure Environment

## Azure 

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell?view=powershell-7.3) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

## Configuration

This notebook performs two tasks:

- Deployment of necessary Azure Services (Azure OpenAI, Azure Cognitive Search) to run samples
- Store all necessary service endpoints, service API keys, Azure OpenAI deployment names in a centralized file (../01_DemoEnvironment/conf/application.env). This file is used by all notebooks in this repo to connect and authenticate against the deployed Azure services.

If you already have instances of Azure OpenAI and Azure Cognitive Search running you can rename the [configuration template](../conf/application.env.example) to `application.env` and provide endpoint, API key and deployment names of a chat completion and an embedding model. We suggest to run the notebook to start a clean environment.

## Step 1:   Login to Azure; Get, Set subscription

In [ ]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

Expected output:

If you're already logged in:
```
    Already logged in as xxxxx
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```
If you aren't logged in a browser window will pop-up which allows you to log in
```
    Logging in...
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```

## Step 2:   Define project unifier

The project unifier is used to allow multiple deployments of services which have a need for a unique custom endpoint.

In [ ]:
$random = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $random"

Expected output:

```
Unifier set to: xxx
```

## Step 3:   Create Resource Group

In this sample all resources are deployed to `eastus`. Feel free to change to your preferred location.

In [ ]:
$resourceGroup = "OpenAI.SKit"
$location = "westus"

az group create `
    --location $location `
    --resource-group $resourceGroup

Expected output: 

 `JSON string describing the newly created resource group`

## Step 4:   Create Azure OpenAI instance

An instance of Azure Cognitive Service with the kind `OpenAI` will be created. The `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csOpenAIName = "SKit$random"

az cognitiveservices account create `
    --name $csOpenAIName `
    --resource-group $resourceGroup `
    --location $location `
    --kind OpenAI `
    --sku S0 `
    --yes

$csOpenAIEndpoint = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csOpenAIApiKey = (
    az cognitiveservices account keys list `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created Azure OpenAI instance`

## Step 5:   Deploy Azure OpenAI models

Two LLM models are deployed to the newly created Azure Cognitive Service instance: 

- A chat completion model. In the sample we're deploying `gpt-35-turbo`. This can be replaced with other models providing a chat completion interface like `gpt-4`.
- A text embedding model. In the sample we're deploying `text-embedding-ada-002`. Any other text embedding model can be deployed as well.

In [ ]:
# Chat Completion Model
$modelChatCompletionDeploymentName = "gpt35turbo"
$modelName = "gpt-35-turbo"
$modelVersion = "1106"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelChatCompletionDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# Text Embedding Model
$modelEmbeddingDeploymentName = "textembeddingada002"
$modelName = "text-embedding-ada-002"
$modelVersion = "2"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelEmbeddingDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# GPT-4 Vision Model
$modelVisionDeploymentName = "gpt4vision"
$modelName = "gpt-4"
$modelVersion = "vision-preview"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelVisionDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

Expected output: 

 `JSON string describing the newly deployed models`

## Step 6:   Create Azure Cognitive Search

Azure Cognitive Search is deployed to use its [vector DB functionalities](https://learn.microsoft.com/en-us/azure/search/vector-search-overview). Just like with Azure OpenAI Cognitive Service, the `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csSearchName = "skitsearch$random"
$csSearchSku = "standard"

az search service create `
    --name $csSearchName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $csSearchSku

$csSearchEndpoint = "https://$csSearchName.search.windows.net"

$csSearchApiKey = ( `
    az search admin-key show `
        --resource-group $resourceGroup `
        --service-name $csSearchName `
        --query primaryKey `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created cognitive search resource`

## Step 7:   Create Azure Storage account

Azure Storage is deployed to store data that can be used to generate indexes in Azure Cognitive Search.

In [ ]:
$stgName = "skitstg$random"
$stgSku = "Standard_LRS"

az storage account create `
    --name $stgName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $stgSku `
    --kind StorageV2 `
    --https-only true `
    --access-tier Hot

$stgConnectionString = ( `
    az storage account show-connection-string `
        --name $stgName `
        --resource-group $resourceGroup `
        --query connectionString `
        --output tsv `
)

Expected output: 

`JSON string describing the newly created storage account`

## Step 8: Set environment variables & create application.env file

In [ ]:
# Set environment variables
$ENV:SKIT_AOAI_APIKEY = $csOpenAIApiKey
$ENV:SKIT_AOAI_ENDPOINT = $csOpenAIEndpoint
$ENV:SKIT_CHATCOMPLETION_DEPLOYMENTNAME = $modelChatCompletionDeploymentName
$ENV:SKIT_EMBEDDINGDEPLOYMENTNAME = $modelEmbeddingDeploymentName
$ENV:SKIT_SEARCH_ENDPOINT = "https://$csSearchEndpoint/"
$ENV:SKIT_SEARCH_APIKEY = $csSearchApiKey
$ENV:SKIT_ASSET_FOLDER = "../../../../assets"
$ENV:SKIT_AOAI1106_APIKEY = $csOpenAIApiKey
$ENV:SKIT_AOAI1106_ENDPOINT = $csOpenAIEndpoint
$ENV:SKIT_AOAI1106_DEPLOYMENTNAME = $modelChatCompletionDeploymentName
$ENV:SKIT_AOAIVISION_APIKEY = $csOpenAIApiKey
$ENV:SKIT_AOAIVISION_ENDPOINT = $csOpenAIEndpoint
$ENV:SKIT_AOAIVISION_DEPLOYMENTNAME = $modelVisionDeploymentName
$ENV:SKIT_STORAGE_CONNECTIONSTRING = $stgConnectionString
Write-Host "Environment variables set!"

$configurationFile = "conf/application.env"

function Set-ConfigurationFileVariable($configurationFile, $variableName, $variableValue) {
    if (Select-String -Path $configurationFile -Pattern $variableName) {
        (Get-Content $configurationFile) | Foreach-Object {
            $_ -replace "$variableName = .*", "$variableName = $variableValue"
        } | Set-Content $configurationFile
    } else {
        Add-Content -Path $configurationFile -value "$variableName = $variableValue"
    }
}

Set-ConfigurationFileVariable $configurationFile "SKIT_AOAI_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAI_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "SKIT_CHATCOMPLETION_DEPLOYMENTNAME" $modelChatCompletionDeploymentName
Set-ConfigurationFileVariable $configurationFile "SKIT_EMBEDDING_DEPLOYMENTNAME" $modelEmbeddingDeploymentName
Set-ConfigurationFileVariable $configurationFile "SKIT_SEARCH_ENDPOINT" $csSearchEndpoint
Set-ConfigurationFileVariable $configurationFile "SKIT_SEARCH_APIKEY" $csSearchApiKey
Set-ConfigurationFileVariable $configurationFile "SKIT_ASSET_FOLDER" "../../../../assets"
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAI1106_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAI1106_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAI1106_DEPLOYMENTNAME" $modelChatCompletionDeploymentName
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAIVISION_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAIVISION_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "SKIT_AOAIVISION_DEPLOYMENTNAME" $modelVisionDeploymentName
Set-ConfigurationFileVariable $configurationFile "SKIT_STORAGE_CONNECTIONSTRING" $stgConnectionString

Write-Host "Configuration file created at: $configurationFile"

Expected output:

```
Environment variables set!
Configuration file created at: xxxxxxxxxxxx
```

## Next Steps

Understand the capabilities of the Azure OpenAI REST API by following the [next notebook](../../03_REST_API/Notebook/01_BasicChat.ipynb).